In [1]:
# Import libraries
import pandas as pd
import numpy as np
import math
import sys
import matplotlib as mpl
import seaborn as sns
import sklearn
from sklearn.manifold import MDS
import matplotlib.pyplot as plt

In [2]:
# Read raw dataset directly from CSSEGISandData repository 

url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
df = pd.read_csv(url)

In [9]:
df.columns

Index(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       ...
       '7/16/22', '7/17/22', '7/18/22', '7/19/22', '7/20/22', '7/21/22',
       '7/22/22', '7/23/22', '7/24/22', '7/25/22'],
      dtype='object', length=920)

In [ ]:
def groupby_country_name(df):

  drop_names = ['Antarctica', 'Diamond Princess', 'MS Zaandam',
              'Summer Olympics 2020', 'West Bank and Gaza',
              'Winter Olympics 2022']

  mask =[]

  for i in range(len(df)):
    if df.iloc[i]['Country/Region'] in drop_names:
      mask.append(i)

  new_df = df.drop(mask, axis=0)

  new_df = new_df.drop(columns=['Province/State', 'Lat', 'Long'])

  df_reduced = new_df.groupby(['Country/Region']).sum()

  df_reduced.reset_index(inplace=True)
  df_reduced = df_reduced.rename(columns = {'index':'Country/Region'})

  return df_reduced

In [ ]:
df_reduced = groupby_country_name(df)

In [ ]:
df_reduced

In [ ]:
# Select the time period of your interest
# For the whole datset, DTW is going to take for ever.

df_reduced.drop(df_reduced.iloc[:, 102:],axis = 1, inplace=True)
df_reduced

In [ ]:
# Shift the time series so day 0 correspond to the first infection for each 
# country 
data = df_reduced.values
time_series = []
for country in data:
  days = [];
  for i in range(1, len(country)):
    if country[i] != 0:
        days.append(country[i])
  time_series.append(days)


In [ ]:
# DTW function: Calculate the distance of two time series

def dtw(s, t):
    n, m = len(s), len(t)
    dtw_matrix = np.zeros((n+1, m+1))

    # Initialization of the dtw matrix
    for i in range(n+1):
        for j in range(m+1):
            dtw_matrix[i, j] = np.inf
    dtw_matrix[0, 0] = 0
    
    # Calculating the elements of the dtw matrix
    for i in range(1, n+1):
        for j in range(1, m+1):
            cost = abs(s[i-1] - t[j-1])
            # take last min from a square box
            last_min = np.min([dtw_matrix[i-1, j], dtw_matrix[i, j-1], dtw_matrix[i-1, j-1]])
            dtw_matrix[i, j] = cost + last_min

    return dtw_matrix[n,m]

In [ ]:
# Progress Bar function

def printProgressBar(i,max,postText):
    n_bar =10 #size of progress bar
    j= i/max
    sys.stdout.write('\r')
    sys.stdout.write(f"[{'=' * int(n_bar * j):{n_bar}s}] {int(100 * j)}%  {postText}")
    sys.stdout.flush()

In [ ]:
# Distance matrix for the countries 
n = len(data)
m = (n,n)
distance_matrix = np.zeros(m)
print("This is going to take a while to calculate.")
for i in range(n):
    for j in range(n):
        # Calculating the distances of the lower triangular matrix because the 
        # distances of the upper triangular can be directly calculated from the 
        # respective distances of the lower triangular
        if(i < j):  
            distance_matrix[i][j] = dtw(time_series[i],time_series[j])
        else:
            distance_matrix[i][j] = distance_matrix[j][i]
    printProgressBar(i,n,"Progress")

In [ ]:
distance_matrix

In [ ]:
distance_matrix_corrected = distance_matrix
distance_matrix_corrected

In [ ]:
# Replace Inf values with max and NaN with 0.0

max = 0.0

for i in range(len(distance_matrix_corrected)):
  for j in range(len(distance_matrix_corrected[i])):
    if distance_matrix_corrected[i,j] > max and distance_matrix_corrected[i,j] != np.inf:
      max = distance_matrix_corrected[i,j]

for i in range(len(distance_matrix_corrected)):
  distance_matrix_corrected[np.isnan(distance_matrix_corrected)] = 0.0
  distance_matrix_corrected[np.isinf(distance_matrix_corrected)] = max

In [ ]:
# Visualization of the countries' distances with MDS

model = MDS(n_components=2, dissimilarity='precomputed', random_state=1)
mds_result = model.fit_transform(distance_matrix_corrected)
x = mds_result[:,0]
y = mds_result[:,1]
plt.scatter(x,y)
plt.ylabel('2nd MDS Component')
plt.xlabel('1st MDS Component')
plt.title('MDS')
plt.show()

In [ ]:
# K-Means clustering with Outliers and Silhouette score

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

best = 0.0 
index = 0  
for cluster_number in range(3,7):
  kmeans = KMeans(n_clusters=cluster_number, random_state=0).fit(mds_result)
  kmeans.labels_
  y_kmeans = kmeans.predict(mds_result)
  silhouette_mean = silhouette_score(mds_result, kmeans.labels_)
  text = ("Average silhouette_score is : %lf" % silhouette_mean)

  plt.figure(figsize=(10,5))
  plt.text( 0, 0 , text, horizontalalignment='right',
           verticalalignment='bottom', weight='bold')
  plt.scatter(x, y, c=y_kmeans, s=50, cmap='viridis')
  plt.ylabel('Second MDS Component')
  plt.xlabel('First MDS Component')
  plt.title('MDS for %d clusters' % cluster_number)
  if(best < silhouette_mean):
    best = silhouette_mean
    index = cluster_number
    best_cluster_with_outliers = y_kmeans

print("The best result was %lf for %d clusters." % (best, index))

In [ ]:
from scipy import stats

# Procedure to find and eliminate outliers 
mds_without_outliers = mds_result[(np.abs(stats.zscore(mds_result)) < 0.5).all(axis=1)]
x1 = mds_without_outliers[:,0]
y1 = mds_without_outliers[:,1]
plt.scatter(x1,y1)
plt.ylabel('2nd MDS Component')
plt.xlabel('1st MDS Component')
plt.title('MDS without outliers')
plt.show()

print("%d countries were removed." % (mds_result.shape[0] - mds_without_outliers.shape[0]))

In [ ]:
# K-Means clustering without Outliers and Silhouette score
best = 0.0 
index = 0  
for cluster_number in range(3,7):
  kmeans = KMeans(n_clusters=cluster_number, random_state=0).fit(mds_without_outliers)
  kmeans.labels_
  y_kmeans1 = kmeans.predict(mds_without_outliers)
  silhouette_mean = silhouette_score(mds_without_outliers, kmeans.labels_)
  text = ("Average silhouette_score is : %lf" % silhouette_mean)
  
  plt.figure(figsize=(10,5))
  plt.text( 0, 0 , text, horizontalalignment='right',
           verticalalignment='bottom', weight='bold')

  plt.scatter(x1, y1, c=y_kmeans1, s=50, cmap='viridis')
  plt.ylabel('Second MDS Component')
  plt.xlabel('First MDS Component')
  plt.title('MDS for %d clusters without outlier' % cluster_number)
  if(best < silhouette_mean):
    best = silhouette_mean
    index = cluster_number
    best_cluster_without_outliers = y_kmeans1
    
print("The best result without outlier was %lf for %d clusters." % (best, index))


In [ ]:
# Add the countries' names in clusters according to the best model regarding silhouette evaluation.
# Best model accoring to silhouette evaluation was for 3 clusters without 
# outlier detection
cluster_A = []
cluster_B = []
cluster_C = []

for i in range(len(data)):
  value = ("%s" % (data[i][0]))
  if(best_cluster_with_outliers[i] == 0):
    cluster_A.append(value)
  elif(best_cluster_with_outliers[i] == 1): 
    cluster_B.append(value)
  else:
    cluster_C.append(value)

In [ ]:
# Print cluster A
cluster_A

In [ ]:
# Print cluster B
cluster_B 

In [ ]:
# Print cluster C
cluster_C